# Use sedona to read various geospatial data format

To work with geospatial data, it's essential to read and write data in geospatial data format. The **full list of the constructor for the geo data types** can be found [here](https://sedona.apache.org/1.6.1/api/sql/Constructor/)
For sedona version `1.6.1`, it supports at least 10 formats:

In this tutorial, we will show example of sedona to read various geospatial data format such as:
- geojson
- shape file
- csv/tsv
- pbf
- geoparquet

## Geospatial data 

To be able to use sedona to do geospatial operations (e.g calculate distance, area hierarchy, etc.), we need to construct geo dataframe first. A geo dataframe contains one or more columns of below type:
- Point : a point on the map with a (x,y) coordinates
- Line: two point which can form a line
- Polygon: a list of point which can form a polygon

The `geometry column` must be able to express these data types.



We will also evaluate the performance(e.g. storage space, processing speed) of each format

In [1]:
from sedona.spark import *
import geopandas as gpd
from pyspark.sql.functions import trim, col
from pathlib import Path
import json

from ipyleaflet import Map, basemaps, basemap_to_tiles, MarkerCluster, Marker, AwesomeIcon
from ipywidgets import Layout
import numpy as np

In [2]:
# get the project root dir
project_root_dir = Path.cwd().parent.parent
data_dir = f"{project_root_dir}/data"

In [3]:
# build a sedona session (sedona = 1.6.1)
jar_folder = Path(f"{project_root_dir}/jars/sedona-35-213-161")
jar_list = [str(jar) for jar in jar_folder.iterdir() if jar.is_file()]
jar_path = ",".join(jar_list)

# build a sedona session (sedona = 1.6.1) offline
config = SedonaContext.builder() \
    .master("local[*]") \
    .config('spark.jars', jar_path). \
    getOrCreate()



In [4]:
# create a sedona context
sedona = SedonaContext.create(config)
sc = sedona.sparkContext


In [5]:
spark = sedona.getActiveSession()

In [6]:
# this sets the encoding of shape files
sc.setSystemProperty("sedona.global.charset", "utf8")

## 1. Read from plain text string

Sedona can read various plain text geospatial data format
- CSV/TSV
- wkt/wkb (Ewkt/Ekb)
- geojson

In below example, we will read a normal csv file which contains two column x, y. You can notice the content of the csv is `plain text` string.

### 1.1 Point example

In below example, we will construct a geo dataframe which contains a **Point** column

In [7]:
point_file_path = f"{data_dir}/csv/test_points.csv"

# read a normal csv
raw_point_df = sedona.read.format("csv").\
          option("delimiter",",").\
          option("header","false").\
          load(point_file_path)

raw_point_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [8]:
raw_point_df.show(5)

+---+-----+
|_c0|  _c1|
+---+-----+
|1.1|101.1|
|2.1|102.1|
|3.1|103.1|
|4.1|104.1|
|5.1|105.1|
+---+-----+
only showing top 5 rows



In [9]:
# create a temp view
raw_point_df.createOrReplaceTempView("p_raw_table")

In [10]:
# we cast the string type to decimal first, then we use `ST_Point` function to build a geometry column by using the two column in the csv file 
geo_point_df = sedona.sql("select ST_Point(cast(p_raw_table._c0 as Decimal(24,20)), cast(p_raw_table._c1 as Decimal(24,20))) as point from p_raw_table")
geo_point_df.printSchema()

root
 |-- point: geometry (nullable = true)



In [11]:
geo_point_df.show(5,truncate=False)

+-----------------+
|point            |
+-----------------+
|POINT (1.1 101.1)|
|POINT (2.1 102.1)|
|POINT (3.1 103.1)|
|POINT (4.1 104.1)|
|POINT (5.1 105.1)|
+-----------------+
only showing top 5 rows



### 1.2 Line example

To create a line type, we can use the constructor **ST_LineStringFromText (Text:string, Delimiter:char)**. In below example, we can notice it takes a list of gps coordinates, then it returns a geometry column.

In [12]:
geo_line_df = sedona.sql("SELECT ST_LineStringFromText('-74.0428197,40.6867969,-74.0421975,40.6921336,-74.0508020,40.6912794', ',') AS line")
geo_line_df.printSchema()

root
 |-- line: geometry (nullable = true)



In [13]:
geo_line_df.show(5,truncate=False)

+----------------------------------------------------------------------------------+
|line                                                                              |
+----------------------------------------------------------------------------------+
|LINESTRING (-74.0428197 40.6867969, -74.0421975 40.6921336, -74.050802 40.6912794)|
+----------------------------------------------------------------------------------+



### 1.3 Polygon example

We have seen the below example for the section 1. We will use the constructor **ST_GeomFromText()**

In [23]:
us_county_file_path = f"{data_dir}/csv/county_small.tsv"

In [26]:
raw_df = sedona.read.format("csv").option("delimiter", "\t").option("header", "false").load(us_county_file_path)
raw_df.show()

+--------------------+---+---+--------+-----+-----------+--------------------+---+---+-----+----+-----+----+----+----------+--------+-----------+------------+
|                 _c0|_c1|_c2|     _c3|  _c4|        _c5|                 _c6|_c7|_c8|  _c9|_c10| _c11|_c12|_c13|      _c14|    _c15|       _c16|        _c17|
+--------------------+---+---+--------+-----+-----------+--------------------+---+---+-----+----+-----+----+----+----------+--------+-----------+------------+
|POLYGON ((-97.019...| 31|039|00835841|31039|     Cuming|       Cuming County| 06| H1|G4020|NULL| NULL|NULL|   A|1477895811|10447360|+41.9158651|-096.7885168|
|POLYGON ((-123.43...| 53|069|01513275|53069|  Wahkiakum|    Wahkiakum County| 06| H1|G4020|NULL| NULL|NULL|   A| 682138871|61658258|+46.2946377|-123.4244583|
|POLYGON ((-104.56...| 35|011|00933054|35011|    De Baca|      De Baca County| 06| H1|G4020|NULL| NULL|NULL|   A|6015539696|29159492|+34.3592729|-104.3686961|
|POLYGON ((-96.910...| 31|109|00835876|31109| 

In [30]:
raw_poly_df = raw_df.select("_c0","_c6").withColumnRenamed("_c0","county_polygon").withColumnRenamed("_c6","county_name")
raw_poly_df.createOrReplaceTempView("gon_raw_table")
raw_poly_df.show(5)

+--------------------+----------------+
|      county_polygon|     county_name|
+--------------------+----------------+
|POLYGON ((-97.019...|   Cuming County|
|POLYGON ((-123.43...|Wahkiakum County|
|POLYGON ((-104.56...|  De Baca County|
|POLYGON ((-96.910...|Lancaster County|
|POLYGON ((-98.273...| Nuckolls County|
+--------------------+----------------+
only showing top 5 rows



In [29]:
raw_poly_df.printSchema()

root
 |-- county_polygon: string (nullable = true)
 |-- county_name: string (nullable = true)



In [32]:
geo_polygon_df=sedona.sql("select ST_GeomFromText(gon_raw_table.county_polygon) as county_shape, gon_raw_table.county_name from gon_raw_table")
geo_polygon_df.show(5)

+--------------------+----------------+
|        county_shape|     county_name|
+--------------------+----------------+
|POLYGON ((-97.019...|   Cuming County|
|POLYGON ((-123.43...|Wahkiakum County|
|POLYGON ((-104.56...|  De Baca County|
|POLYGON ((-96.910...|Lancaster County|
|POLYGON ((-98.273...| Nuckolls County|
+--------------------+----------------+
only showing top 5 rows



In [33]:
geo_polygon_df.printSchema()

root
 |-- county_shape: geometry (nullable = true)
 |-- county_name: string (nullable = true)



## 1.4 Read wkt and wkb file

**Well-known text (WKT)** is a `text markup language` for representing vector geometry objects on a map and spatial reference systems of spatial objects. A binary equivalent, known as **well-known binary (WKB)** is used to transfer and store the same information for geometry objects.

Geometries in a `WKT and WKB` file always occupy a single column no matter how many coordinates they have. Sedona provides `WktReader and WkbReader` to create generic SpatialRDD. Then we need to convert the spatial rdd to dataframe.

> You must use the wkt reader to read wkt file, and wkb reader to read wkb file.

For `EWKT/EWKB`, we just have a extra column `SRID(Spatial Reference Identifier)  code` compare to WKT

```sql
SELECT ST_AsText(ST_GeomFromEWKT('SRID=4269;POINT(40.7128 -74.0060)'))

# output example
# POINT(40.7128 -74.006)
```

In [34]:
us_county_wkb_file_path = f"{data_dir}/csv/county_small_wkb.tsv"

In [35]:
from sedona.core.formatMapper import WkbReader
from sedona.utils.adapter import Adapter

In [36]:
# The WKT string starts from Column 0
wkbColumn = 0 
allowTopologyInvalidGeometries = True
skipSyntaxInvalidGeometries = False

spatialRdd = WkbReader.readToGeometryRDD(sedona.sparkContext, us_county_wkb_file_path, wkbColumn, allowTopologyInvalidGeometries, skipSyntaxInvalidGeometries)

In [40]:
geo_county_wkb_df = Adapter.toDf(spatialRdd,sedona).withColumnRenamed("geometry", "county_shape")
geo_county_wkb_df.show(5)

+--------------------+
|        county_shape|
+--------------------+
|POLYGON ((-97.019...|
|POLYGON ((-123.43...|
|POLYGON ((-104.56...|
|POLYGON ((-96.910...|
|POLYGON ((-98.273...|
+--------------------+
only showing top 5 rows



In [41]:
geo_county_wkb_df.printSchema()

root
 |-- county_shape: geometry (nullable = true)



### 1.5 Read geojson

https://sedona.apache.org/1.6.1/tutorial/sql
Geojson has two different organization:
- single-line GeoJSON
- multi-line 

#### 1.5.1 Read single-line GeoJSON

In the single-line geoJSON organization, each line is a separate, self-contained GeoJSON object. Below is an example

```json
{"type":"Feature","geometry":{"type":"Point","coordinates":[102.0,0.5]},"properties":{"prop0":"value0"}}
{"type":"Feature","geometry":{"type":"LineString","coordinates":[[102.0,0.0],[103.0,1.0],[104.0,0.0],[105.0,1.0]]},"properties":{"prop0":"value1"}}
{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[100.0,0.0],[101.0,0.0],[101.0,1.0],[100.0,1.0],[100.0,0.0]]]},"properties":{"prop0":"value2"}}
```
You can notice that each line starts with `{` ends with `}`, which means it's a self-contained json object.

> This format is efficient for processing large datasets, because each line is an independent GeoJSON Feature which can be processed in parallel.  

In [14]:
us_county_json_file_path = f"{data_dir}/geojson/us_county.json"

In [15]:
raw_json_df = sedona.read.format("geojson").load(us_county_json_file_path)
raw_json_df.show(5)

+--------------------+--------------------+-------+
|            geometry|          properties|   type|
+--------------------+--------------------+-------+
|POLYGON ((-87.621...|{1500000US0107701...|Feature|
|POLYGON ((-85.719...|{1500000US0104502...|Feature|
|POLYGON ((-86.000...|{1500000US0105500...|Feature|
|POLYGON ((-86.574...|{1500000US0108900...|Feature|
|POLYGON ((-85.382...|{1500000US0106904...|Feature|
+--------------------+--------------------+-------+
only showing top 5 rows



In [16]:
raw_json_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- AFFGEOID: string (nullable = true)
 |    |-- ALAND: long (nullable = true)
 |    |-- AWATER: long (nullable = true)
 |    |-- BLKGRPCE: string (nullable = true)
 |    |-- COUNTYFP: string (nullable = true)
 |    |-- GEOID: string (nullable = true)
 |    |-- LSAD: string (nullable = true)
 |    |-- NAME: string (nullable = true)
 |    |-- STATEFP: string (nullable = true)
 |    |-- TRACTCE: string (nullable = true)
 |-- type: string (nullable = true)



### 1.5.2 Read multi-line GeoJSON

The multi-line GeoJSON use a global `{ "type": "FeatureCollection", }` to encapsulate all geo features in one JSON object. Below is an example

```json
{ "type": "FeatureCollection",
    "features": [
      { "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [102.0, 0.5]},
        "properties": {"prop0": "value0"}
        },
      { "type": "Feature",
        "geometry": {
          "type": "LineString",
          "coordinates": [
            [102.0, 0.0], [103.0, 1.0], [104.0, 0.0], [105.0, 1.0]
            ]
          },
        "properties": {
          "prop0": "value1",
          "prop1": 0.0
          }
        },
      { "type": "Feature",
         "geometry": {
           "type": "Polygon",
           "coordinates": [
             [ [100.0, 0.0], [101.0, 0.0], [101.0, 1.0],
               [100.0, 1.0], [100.0, 0.0] ]
             ]
         },
         "properties": {
           "prop0": "value2",
           "prop1": {"this": "that"}
           }
         }
       ]
}
```

Multiline format is preferable for scenarios where files need to be human-readable or manually edited.

As the entire file is considered as a single json object, it's hard to process in parallel

In [24]:
from pyspark.sql.functions import expr

multi_line_json_file_path = f"{data_dir}/geojson/multi_lines.json"

df_raw = sedona.read.format("geojson").option("multiLine", "true").load(multi_line_json_file_path)
          
df_raw.show(5,truncate=False)
df_raw.printSchema()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|features                                                                                                                                                                                            |type             |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|[{POINT (102 0.5), {value0, NULL}, Feature}, {LINESTRING (102 0, 103 1, 104 0, 105 1), {value1, 0.0}, Feature}, {POLYGON ((100 0, 101 0, 101 1, 100 1, 100 0)), {value2, {"this":"that"}}, Feature}]|FeatureCollection|
+-----------------------------------------------------------------------------------------------------------------------------------

As the entire file is a single json object(FeatureCollection), all features are consider as items in one big array, To get one feature per row, we need to explode the array envelope.

In [23]:
df = df_raw.selectExpr("explode(features) as features").select("features.*")
 

df.show()
df.printSchema()

+--------------------+--------------------+-------+
|            geometry|          properties|   type|
+--------------------+--------------------+-------+
|     POINT (102 0.5)|      {value0, NULL}|Feature|
|LINESTRING (102 0...|       {value1, 0.0}|Feature|
|POLYGON ((100 0, ...|{value2, {"this":...|Feature|
+--------------------+--------------------+-------+

root
 |-- geometry: geometry (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- prop0: string (nullable = true)
 |    |-- prop1: string (nullable = true)
 |-- type: string (nullable = true)



## 1.6 Read GML

**GML(Geography Markup Language)** is an `XML based encoding standard` for geographic information developed by the `OpenGIS Consortium (OGC)`. You can find the official doc [here](https://www.ogc.org/publications/standard/gml/)
It has three major version:
- GML 1
- GML 2
- GML 3

Sedona(<v1.6.1) only supports `GML1 and GML2` for now.  

In [33]:
gml_sample = """
<gml:LineString srsName="EPSG:4269">
        <gml:coordinates>
            -71.16028,42.258729
            -71.160837,42.259112
            -71.161143,42.25932
        </gml:coordinates>
</gml:LineString>
"""

gml_df = sedona.sql(f"SELECT ST_GeomFromGML('{gml_sample}') as line")

In [35]:
gml_df.show(5, truncate=False)
gml_df.printSchema()

+---------------------------------------------------------------------------+
|line                                                                       |
+---------------------------------------------------------------------------+
|LINESTRING (-71.16028 42.258729, -71.160837 42.259112, -71.161143 42.25932)|
+---------------------------------------------------------------------------+

root
 |-- line: geometry (nullable = true)



## 1.7 Read KML

**Keyhole Markup Language (KML)** is an `XML notation` for expressing geographic annotation and visualization within two-dimensional maps and three-dimensional Earth browsers. `KML was developed for use with Google Earth`, which was originally named `Keyhole Earth Viewer`.

A complete kml file example.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
<Document>
<Placemark>
  <name>New York City</name>
  <description>New York City</description>
  <Point>
    <coordinates>-74.006393,40.714172,0</coordinates>
  </Point>
</Placemark>
</Document>
</kml>
``` 

You can notice the coordinates has three attributes (longitude,latitude,altitude)





### Disk usage

The shape file use 301 MB disk space

In [31]:
! du -ah /home/pengfei/data_set/kaggle/geospatial/communes_fr_shape


8.9M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-20220101.dbf
4.0K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-20220101.prj
4.0K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-descriptif.txt
276K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-20220101.shx
4.0K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/LICENCE.txt
4.0K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-20220101.cpg
292M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape/communes-20220101.shp
301M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_shape


In [5]:
from pyspark.sql import DataFrame


def get_nearest_commune(df:DataFrame, latitude:str, longitude:str, max_commune_number:int):
    temp_table_name:str = "temp_tab"
    df.createOrReplaceTempView(temp_table_name)
    nearest_commune_df = sedona.sql(f"""
     SELECT z.nom as commune_name, z.insee, ST_DistanceSphere(ST_PointFromText('{longitude},{latitude}', ','), z.geometry) AS distance FROM {temp_table_name} as z ORDER BY distance ASC LIMIT {max_commune_number}
     """)
    return nearest_commune_df

In [6]:
# the gps coordinates for kremlin-Bicetre is 48.8100° N, 2.3539° E

kb_latitude = "48.8100"
kb_longitude = "2.3539"

In [7]:

kb_nearest_shape_df = get_nearest_commune(fr_commune_df,kb_latitude,kb_longitude,10)

In [8]:
%%time

kb_nearest_shape_df.show()
kb_nearest_shape_df.count()

+------------------+-----+------------------+
|      commune_name|insee|          distance|
+------------------+-----+------------------+
|Le Kremlin-Bicêtre|94043|198.60307108585405|
|          Gentilly|94037| 798.3521490770968|
|           Arcueil|94003|1543.0937442695515|
|         Villejuif|94076| 2007.793912679607|
|    Ivry-sur-Seine|94041| 2489.634383841373|
|            Cachan|94016| 2590.828517555236|
|         Montrouge|92049| 2750.714176859015|
|           Bagneux|92007| 3462.091511432535|
|   Vitry-sur-Seine|94081|3845.1624363327196|
|   L'Haÿ-les-Roses|94038| 3942.190017739479|
+------------------+-----+------------------+

CPU times: total: 0 ns
Wall time: 3.13 s


10

## Read write GeoParquet
GeoParquet is an **incubating Open Geospatial Consortium (OGC) standard** that adds interoperable geospatial types `(Point, Line, Polygon)` to Parquet. Currently(16/04/2024), the stable version is 1.0.0
You can find the official site of geo-parquet [here](https://geoparquet.org/)

In [7]:
clean_fr_commune_df = fr_commune_df.withColumn("clean_nom",trim(col("nom"))).withColumn("clean_insee",trim(col("insee"))).drop("nom").drop("insee").withColumnRenamed("clean_nom","nom").withColumnRenamed("clean_insee","insee")

In [8]:
clean_fr_commune_df.show()

+--------------------+--------------------+--------------------+-----------------+-----+
|            geometry|           wikipedia|             surf_ha|              nom|insee|
+--------------------+--------------------+--------------------+-----------------+-----+
|POLYGON ((9.32016...|fr:Pie-d'Orezza  ...|     573.00000000...|     Pie-d'Orezza|2B222|
|POLYGON ((9.20010...|fr:Lano          ...|     824.00000000...|             Lano|2B137|
|POLYGON ((9.27757...|fr:Cambia        ...|     833.00000000...|           Cambia|2B051|
|POLYGON ((9.25119...|fr:Érone         ...|     393.00000000...|            Érone|2B106|
|POLYGON ((9.28339...|fr:Oletta        ...|    2674.00000000...|           Oletta|2B185|
|POLYGON ((9.30951...|fr:Canari (Haute-...|    1678.00000000...|           Canari|2B058|
|POLYGON ((9.30101...|fr:Olmeta-di-Tuda...|    1753.00000000...|   Olmeta-di-Tuda|2B188|
|POLYGON ((9.32662...|fr:Campana       ...|     236.00000000...|          Campana|2B052|
|POLYGON ((9.33944...

In [9]:
fr_commune_geoparquet_file_path = "/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet"
clean_fr_commune_df.write.format("geoparquet").option("geoparquet.version","1.0.0").save(fr_commune_geoparquet_file_path)

In [10]:
! du -ah /home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet

0	/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet/_SUCCESS
2.3M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet/.part-00000-82765f1e-fe4e-4e74-81e2-01fd73bcdb34-c000.snappy.parquet.crc
4.0K	/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet/._SUCCESS.crc
291M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet/part-00000-82765f1e-fe4e-4e74-81e2-01fd73bcdb34-c000.snappy.parquet
294M	/home/pengfei/data_set/kaggle/geospatial/communes_fr_geoparquet


In [11]:
geo_parquet_df = sedona.read.format("geoparquet").load(fr_commune_geoparquet_file_path)

In [12]:
geo_parquet_df.show()
geo_parquet_df.count()

+--------------------+--------------------+--------------------+-----------------+-----+
|            geometry|           wikipedia|             surf_ha|              nom|insee|
+--------------------+--------------------+--------------------+-----------------+-----+
|POLYGON ((9.32016...|fr:Pie-d'Orezza  ...|     573.00000000...|     Pie-d'Orezza|2B222|
|POLYGON ((9.20010...|fr:Lano          ...|     824.00000000...|             Lano|2B137|
|POLYGON ((9.27757...|fr:Cambia        ...|     833.00000000...|           Cambia|2B051|
|POLYGON ((9.25119...|fr:Érone         ...|     393.00000000...|            Érone|2B106|
|POLYGON ((9.28339...|fr:Oletta        ...|    2674.00000000...|           Oletta|2B185|
|POLYGON ((9.30951...|fr:Canari (Haute-...|    1678.00000000...|           Canari|2B058|
|POLYGON ((9.30101...|fr:Olmeta-di-Tuda...|    1753.00000000...|   Olmeta-di-Tuda|2B188|
|POLYGON ((9.32662...|fr:Campana       ...|     236.00000000...|          Campana|2B052|
|POLYGON ((9.33944...

34955

In [18]:
kb_nearest_parquet_df = get_nearest_commune(geo_parquet_df,kb_latitude,kb_longitude,10)

In [19]:
%%time
kb_nearest_parquet_df.show()
kb_nearest_parquet_df.count()

+------------------+-----+------------------+
|      commune_name|insee|          distance|
+------------------+-----+------------------+
|Le Kremlin-Bicêtre|94043|255.77950075329835|
|          Gentilly|94037| 1138.204118880015|
|         Villejuif|94076|2067.5242470555963|
|           Arcueil|94003| 2269.505672821453|
|            Cachan|94016|3169.7694895288837|
|    Ivry-sur-Seine|94041| 3769.348960915047|
|         Montrouge|92049| 4124.301376321017|
|   L'Haÿ-les-Roses|94038| 4166.688028197553|
|    Chevilly-Larue|94021| 4789.020724647998|
|           Bagneux|92007|  5041.99634269013|
+------------------+-----+------------------+


CPU times: user 9.98 ms, sys: 6.56 ms, total: 16.5 ms
Wall time: 6.6 s


10

### Custom metadata in geo parquet

Compare the result of shape file and geo parquet, we don't gain too many things

| file format | disk space | distance (in sec) |
|-------------|------------|-------------------|
| shape file  | 301        | 7,45              |
| geoparquet  | 294        | 6,60              |

## Read write GeoJSON(Geographic JavaScript Object Notation)

Sedona can read geojson easily, but can't write geojson. Geo pandas can write geojson. But it can't support large 
data frame. Below are two examples. In the first, we create a simple geo dataframe. It works without problem.
The second does work at all. We have an oom error.

In [37]:
from shapely import Point

data = {
    'id': [1, 2, 3],
    'name': ['A', 'B', 'C'],
    'geometry': [Point(1, 1), Point(2, 2), Point(3, 3)]
}

fr_commune_geoj_file_path = "/home/pengfei/data_set/kaggle/geospatial/communes_fr_geojson.json"

gdf = gpd.GeoDataFrame(data, crs="EPSG:4326")

print(gdf.head())

# Write GeoDataFrame to GeoJSON file
gdf.to_file(fr_commune_geoj_file_path, driver='GeoJSON')

   id name                 geometry
0   1    A  POINT (1.00000 1.00000)
1   2    B  POINT (2.00000 2.00000)
2   3    C  POINT (3.00000 3.00000)


In [35]:
fr_commune_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            geometry|               insee|                 nom|           wikipedia|             surf_ha|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|POLYGON ((9.32016...|2B222            ...|Pie-d'Orezza     ...|fr:Pie-d'Orezza  ...|     573.00000000...|
|POLYGON ((9.20010...|2B137            ...|Lano             ...|fr:Lano          ...|     824.00000000...|
|POLYGON ((9.27757...|2B051            ...|Cambia           ...|fr:Cambia        ...|     833.00000000...|
|POLYGON ((9.25119...|2B106            ...|Érone            ...|fr:Érone         ...|     393.00000000...|
|POLYGON ((9.28339...|2B185            ...|Oletta           ...|fr:Oletta        ...|    2674.00000000...|
|POLYGON ((9.30951...|2B058            ...|Canari           ...|fr:Canari (Haute-...|    1678.00000000...|
|POLYGON ((9.30101...|2B188          

In [23]:
from shapely import Polygon
from pyspark.sql.functions import collect_list


def get_geopandas_df(spark_df:DataFrame):
    # Convert Spark DataFrame to Pandas DataFrame
    pandas_df = spark_df.toPandas()

    # Create a GeoPandas DataFrame from the Pandas DataFrame
    # Make sure to create Shapely geometry objects from the geometry column
    pandas_df['geometry'] = pandas_df['geometry'].apply(lambda x: Polygon(eval(x)))
    geo_df = gpd.GeoDataFrame(pandas_df, geometry='geometry')
    
    return geo_df

In [24]:
gdf = get_geopandas_df(fr_commune_df)

24/04/15 16:03:16 ERROR Executor: Exception in task 0.0 in stage 21.0 (TID 19)
org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 67108864. To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:391)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:593)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: com.esotericsoftware.kryo.KryoException: Buffer overflow. Available: 0, required: 67108864
	at com.esotericsoftware.kryo.io.Output.require(Output.java:167)
	at com.esotericsoftware.kryo.io.Output.writeBytes(Output.java:251)
	at com.esotericsoftware.kryo.io.Output.writeBytes(Output.java:237)
	at com.esotericsoftware.kryo.serialize

Py4JJavaError: An error occurred while calling o49.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 19) (10.50.2.80 executor driver): org.apache.spark.SparkException: Kryo serialization failed: Buffer overflow. Available: 0, required: 67108864. To avoid this, increase spark.kryoserializer.buffer.max value.
	at org.apache.spark.serializer.KryoSerializerInstance.serialize(KryoSerializer.scala:391)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:593)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)
Caused by: com.esotericsoftware.kryo.KryoException: Buffer overflow. Available: 0, required: 67108864
	at com.esotericsoftware.kryo.io.Output.require(Output.ja

In [39]:
city_geoj_file_path = "/home/pengfei/data_set/kaggle/geospatial/world-cities.geojson"

# read geo json
# the selectExpr action Explode the envelope to get one feature per row.
#  Unpack the features' struct. 
df = sedona.read.format("json").option("multiLine", "true").load(city_geoj_file_path) \
 .selectExpr("explode(features) as features")  \
 .select("features.*")  
 # .withColumn("prop0", f.expr("properties['prop0']")).drop("properties").drop("type")

df.show()
df.printSchema()

+--------------------+--------------------+-------+
|            geometry|          properties|   type|
+--------------------+--------------------+-------+
|{[121.4961111, 25...|{Yungho, yungho, ...|Feature|
|{[-72.233333, -37...|{Mulchen, mulchen...|Feature|
|{[-73.6405556, 40...|{Oceanside, ocean...|Feature|
|{[-70.966667, -32...|{Llaillay, llaill...|Feature|
|{[35.6, 3.1166667...|{Lodwar, lodwar, ...|Feature|
|{[10.1666667, 5.9...|{Bamenda, bamenda...|Feature|
|{[-45.533333, -20...|{Arcos, arcos, br...|Feature|
|{[-43.716944, -22...|{Seropédica, sero...|Feature|
|{[-97.1413889, 32...|{Mansfield, mansf...|Feature|
|{[-67.5419444, 10...|{Palo Negro, palo...|Feature|
|{[-42.683333, -5....|{Demerval Lobão, ...|Feature|
|{[-48.666667, -28...|{Imbituba, imbitu...|Feature|
|{[-49.333333, -5....|{Itupiranga, itup...|Feature|
|{[121.75, 24.7666...|{Ilan, ilan, tw, ...|Feature|
|{[49.1825, 11.284...|{Bosaso, bosaso, ...|Feature|
|{[64.570048, 31.8...|{Geresk, geresk, ...|Feature|
|{[7.573271,